In [106]:
!pip install transformers

In [154]:
import json
import requests
import time
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

In [108]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd


# Postgres vector search
We have seen that postgres is capable of running a k-nearest neighbor search with the cosine similarity metric

## Example
These are the 10 closest apps to the zero vector

In [109]:
vec = [0 for _ in range(1024)]
sql_query = f'''
SELECT
    *
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
LIMIT 10
'''

with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.31683657,0.6250096,-0.12035612,0.21742296,-..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.54112995,0.74467605,-0.58174556,0.9596781,0..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176758,0.6059474,-1.139566,0.07630952,-0...."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.05092397,1.0967331,-0.30831552,0.13220598,-..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.8975636,0.4644263,-0.1801914,1.2617054,0.07..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490415,0.6585203,-0.33866996,0.18323657,0..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.20005348,0.3608348,-0.137923,0.34308633,-0...."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.51124793,0.512806,-0.056304544,1.2819325,0...."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094386,0.41121805,-0.6196006,0.5350147,0...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.20005348,0.3608348,-0.137923,0.34308633,-0...."


In [110]:
# Embedding new data
We can embed new data with this API call

SyntaxError: invalid syntax (2159921380.py, line 2)

In [ ]:
query_text = "A farming game"
query_vec = requests.get("http://emb:8080/embed", params ={"text": query_text}).json()
print(query_vec)

#  Embed the text data within Postgres?
You are were asked to extend Postgres, such that the following syntax would work
## Desired outcome

    SELECT
        "bundleId"
    FROM app_vectors
    ORDER BY embedding<=>embed('a farming game')
    LIMIT 10

## Steps to achieve this
1. Use the [pghttp](https://github.com/pramsey/pgsql-http) extension to query the embedding endpoint.
2. Create an [SQL Function](https://www.postgresql.org/docs/current/sql-createfunction.html).
3. Verify that the above `embed` function yields the same result as embedding in python
4. Verify that the `ORDER BY embedding<=>embed('a farming game')` clause works without erros.

## Submission
You have 3 hours to complete the assignment.

Feel free to use any tool you think is helpful (Google, ChatGPT, CoPilot, etc).


Once done please notify your interviewer.


## Uri's Number:
0507-649-650

In [124]:
import json
import requests
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Database connection
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Example embedding vector
vec = [0 for _ in range(1024)]
sql_query = f'''
SELECT
    *
FROM app_vectors
ORDER BY embedding <=> '{json.dumps(vec)}'
LIMIT 10
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

print(df)

# Embedding new data
query_text = "A farming game"
query_vec = requests.get("http://emb:8080/embed", params={"text": query_text}).json()
print(query_vec)




   id                     bundleId  \
0   1             com.loop.match3d   
1   2       com.YayySAL.DodgeAgent   
2   3        com.tintash.nailsalon   
3   4        com.tilegarden.match3   
4   6   com.AppIdeas.LevelUpRunner   
5  17                   1582745578   
6  18                   1569586264   
7   5  com.kamilbilge.ropesavior3d   
8   7               se.ace.fishinc   
9   8      com.volt.dresstoimpress   

                                             content  \
0  Get ready for a new, challenging and original ...   
1  Your mission, should you choose to accept it, ...   
2  It is manicure madness over here and it�s your...   
3  \nIf you enjoy playing Match 3 tile & mahjong ...   
4  Are you a real hero? Are you the strongest, th...   
5  Dominate the coal industry and become the next...   
6  Choose the appropriate outfit to make it throu...   
7  Become the hero we need in this tangled puzzle...   
8  Gather your riches and travel across the open ...   
9  \nChoose the appro

In [160]:
from sqlalchemy import text

sql_fun = '''
CREATE OR REPLACE FUNCTION embed(text)
RETURNS json AS $$
DECLARE
    result json;
BEGIN
    SELECT content::json
    INTO result
    FROM http_get('http://emb:8080/embed?text=' || $1);

    RETURN result;
END;
$$ LANGUAGE plpgsql;
'''


with engine.connect() as db_con:
    res = db_con.execute(text(sql_fun))
    


In [141]:
from sqlalchemy import text

sql_fun = '''
CREATE OR REPLACE FUNCTION embed(text)
RETURNS json AS $$
DECLARE
    result json;
BEGIN
    SELECT content::json
    INTO result
    FROM http_get('http://emb:8080/embed?text=a%20farming%20game');

    RETURN result;
END;
$$ LANGUAGE plpgsql;
'''

sql_check = '''
SELECT embed('123');
'''

with engine.connect() as db_con:
    res = db_con.execute(text(sql_fun))
    result = db_con.execute(text(sql_check))
    rows = result.fetchall()
    
    # Convert the result to a pandas DataFrame for better display
    df = pd.DataFrame(rows, columns=result.keys())
df


,embed
0,"[0.354161, 0.84812, -0.030283, -0.266633, -0.2..."


In [150]:
from sqlalchemy import text

sql_fun = '''
CREATE OR REPLACE FUNCTION embed(input_text text)
RETURNS json AS $$
DECLARE
    result json;
    transformed_text text;
BEGIN
    transformed_text := REPLACE(input_text, ' ', '%20');
    SELECT content::json
    INTO result
    FROM http_get('http://emb:8080/embed?text=' || transformed_text);

    RETURN result;
END;
$$ LANGUAGE plpgsql;
'''

sql_check = '''
SELECT embed('a farming game');
'''


with engine.connect() as db_con:
    res = db_con.execute(text(sql_fun))
    result = db_con.execute(text(sql_check))
    rows = result.fetchall()
    
    # Convert the result to a pandas DataFrame for better display
    df = pd.DataFrame(rows, columns=result.keys())
df


,embed
0,"[0.354161, 0.84812, -0.030283, -0.266633, -0.2..."


In [158]:
from sqlalchemy import create_engine, text
import pandas as pd

# Define the function creation SQL statement
sql_fun = '''
CREATE OR REPLACE FUNCTION embed(input_text text)
RETURNS vector AS $$
DECLARE
    result vector;
    json_result json;
    transformed_text text;
BEGIN
    transformed_text := REPLACE(input_text, ' ', '%20');
    
    -- Make the HTTP request with the transformed text
    SELECT content::json
    INTO json_result
    FROM http_get('http://emb:8080/embed?text=' || transformed_text);

    -- Assuming the JSON response contains a key 'vector' which holds the array
    SELECT array_agg(value::numeric)
    INTO result
    FROM json_array_elements_text(json_result->'vector') AS value;

    RETURN result;
END;
$$ LANGUAGE plpgsql;
'''

# Define the function check SQL statement
sql_check_final = '''
SELECT
    *
FROM app_vectors
ORDER BY embedding <=> embed('a farming game')
LIMIT 10
'''

with engine.connect() as db_con:
    # Execute the function creation
    db_con.execute(text(sql_fun))
    
    # Execute the function check
    result = db_con.execute(text(sql_check_final))
    
    # Fetch all rows from the result
    rows = result.fetchall()
    
    # Convert the result to a pandas DataFrame for better display
    df = pd.DataFrame(rows, columns=result.keys())

# Display the DataFrame
df


,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.31683657,0.6250096,-0.12035612,0.21742296,-..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.54112995,0.74467605,-0.58174556,0.9596781,0..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176758,0.6059474,-1.139566,0.07630952,-0...."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.05092397,1.0967331,-0.30831552,0.13220598,-..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.8975636,0.4644263,-0.1801914,1.2617054,0.07..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490415,0.6585203,-0.33866996,0.18323657,0..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.20005348,0.3608348,-0.137923,0.34308633,-0...."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.51124793,0.512806,-0.056304544,1.2819325,0...."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094386,0.41121805,-0.6196006,0.5350147,0...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.20005348,0.3608348,-0.137923,0.34308633,-0...."


In [159]:
from sqlalchemy import create_engine, text
import pandas as pd

# Define the function creation SQL statement
sql_fun = '''
CREATE OR REPLACE FUNCTION embed(input_text text)
RETURNS vector AS $$
DECLARE
    result vector;
    json_result json;
    transformed_text text;
BEGIN
    transformed_text := REPLACE(input_text, ' ', '%20');
    
    -- Make the HTTP request with the transformed text
    SELECT content::json
    INTO json_result
    FROM http_get('http://emb:8080/embed?text=' || transformed_text);

    -- Assuming the JSON response contains a key 'vector' which holds the array
    SELECT array_agg(value::numeric)
    INTO result
    FROM json_array_elements_text(json_result->'vector') AS value;

    RETURN result;
END;
$$ LANGUAGE plpgsql;
'''

# Define the function check SQL statement
sql_check_final = '''
SELECT
    *
FROM app_vectors
ORDER BY embedding <=> embed('a farming game')
LIMIT 10
'''

with engine.connect() as db_con:
    # Execute the function creation
    db_con.execute(text(sql_fun))
    
    # Execute the function check
    result = db_con.execute(text(sql_check_final))
    
    # Fetch all rows from the result
    rows = result.fetchall()
    
    # Convert the result to a pandas DataFrame for better display
    df = pd.DataFrame(rows, columns=result.keys())

# Display the DataFrame
df


,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.31683657,0.6250096,-0.12035612,0.21742296,-..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.54112995,0.74467605,-0.58174556,0.9596781,0..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176758,0.6059474,-1.139566,0.07630952,-0...."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.05092397,1.0967331,-0.30831552,0.13220598,-..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.8975636,0.4644263,-0.1801914,1.2617054,0.07..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490415,0.6585203,-0.33866996,0.18323657,0..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.20005348,0.3608348,-0.137923,0.34308633,-0...."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.51124793,0.512806,-0.056304544,1.2819325,0...."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.73094386,0.41121805,-0.6196006,0.5350147,0...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.20005348,0.3608348,-0.137923,0.34308633,-0...."
